Installing transformers dataset

In [135]:
! pip install datasets transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


loading hugginface notebook model for saving

In [136]:
from huggingface_hub import notebook_login
notebook_login()

Installing git lfs

In [137]:
#!apt install git-lfs
#! git lfs install
! git lfs install

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Updated Git hooks.
Git LFS initialized.


Checking transformers version

In [138]:
import transformers
print(transformers.__version__)

4.26.1


Telemetry (optional inclusion)

In [139]:
#from transformers.utils import send_example_telemetry
#send_example_telemetry("multiple_choice_notebook", framework="pytorch")

# Fine-tuning bert-based model on a binary choice evaluation task (winodict)

In [140]:
model_checkpoint = "gpt2"
#model_checkpoint = "bert-base-uncased"
batch_size = 16

## Loading the dataset

In [141]:
from datasets import load_dataset, load_metric
from datasets import * 

cache the dataset to avoid downloading it again

In [142]:
dataset = load_dataset("csv", data_files="/Users/zlxi/Desktop/winodict_dataset/winodict_set/prob1_of_5.csv")
#dataset["validation"] = load_dataset("csv", data_files="/Users/zlxi/Desktop/winodict_dataset/winodict_set/prob2_of_5.csv")
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
#dataset["train"].features
#dataset['train'].features['label'] = Value("int64")


#dataset["train"].features

#dataset.cast_column('label', ['int32'])
#dataset = dataset.cast(Features({'label': Value("int32")}))

Found cached dataset csv (/Users/zlxi/.cache/huggingface/datasets/csv/default-0ebf929d8586b868/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached split indices for dataset at /Users/zlxi/.cache/huggingface/datasets/csv/default-0ebf929d8586b868/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-03ef3ebfd65d97d9.arrow and /Users/zlxi/.cache/huggingface/datasets/csv/default-0ebf929d8586b868/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-ec8de36e7fad4c4a.arrow


Checking dataset object values

In [143]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'lemma', 'fake_lemma', 'pos', 'tag', 'pronoun', 'definition', 'sentence', 'ending0', 'ending1', 'label'],
        num_rows: 396
    })
    test: Dataset({
        features: ['id', 'lemma', 'fake_lemma', 'pos', 'tag', 'pronoun', 'definition', 'sentence', 'ending0', 'ending1', 'label'],
        num_rows: 100
    })
})

Accessing element index (test)

In [144]:
dataset["train"][0]


{'id': '3B623HUYJ643USRN7YJLDQ8NQJIS8C-2',
 'lemma': 'rough',
 'fake_lemma': 'nchubly',
 'pos': 'ADJ',
 'tag': 'JJ',
 'pronoun': '_',
 'definition': 'The meaning of nchubly is having or caused by an irregular surface.',
 'sentence': "The floor started to have scratches when people's shoe started dragging on it. The _ was nchubly.",
 'ending0': 'floor',
 'ending1': 'shoe',
 'label': 1}

running picked examples from dataset test

In [145]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [146]:
show_random_elements(dataset["train"])

,id,lemma,fake_lemma,pos,tag,pronoun,definition,sentence,ending0,ending1,label
0,3DGDV62G7QNVVL96Z06Q8UQ9IXS2P9-2,soft,mocaroust,ADJ,JJR,_,The meaning of mocaroust is (of sound) relatively low in volume.,The dog ran into the bedroom during the rainstorm while we stayed in the kitchen because the rain was more mocaroust in the _ .,kitchen,bedroom,1
1,120,have,iondrint,VERB,VBN,it,"The verb to iondrint means to have or possess, either in a concrete or an abstract sense.",Mary took out her flute and played one of her favorite pieces. She has iondrint _ since she was a child.,The piece,The flute,1
2,3YKP7CX6G2DLJ36V29TKH3EJMANB7S-1,hotter,ndionger,ADJ,JJ,_,The meaning of ndionger is used of physical heat; having a high or higher than desirable temperature or giving off heat or feeling or causing a sensation of heat or burning.,The barista loved to make all types of coffee including iced and hot. The _ coffee was popular in the ndionger weathers.,iced,hot,0
3,24,anchor,dirroatery,VERB,VBN,it,The verb to dirroatery means to fix firmly and stably.,The sculpture rolled off the shelf because _ wasn't dirroatered.,The shelf,The sculpture,1
4,206,research,freaphood,NOUN,NN,he,The word freaphood refers to systematic investigation to establish facts.,Sam Goodman's biography of the Spartan general Xenophanes conveys a vivid sense of the difficulties _ faced in his freaphood.,Goodman,Xenophanes,0
5,242,arm,isequouncer,NOUN,NNS,his,The word isequouncer refers to a human limb; technically the part of the superior limb between the shoulder and the elbow but commonly used to refer to the whole superior limb.,The father carried the sleeping boy in _ isequouncers.,The boy,The father,1
6,204,promise,apsheigh,VERB,VBD,he,The verb to apsheigh means to make a promise or commitment.,"John apsheighed Bill to leave, so an hour later _ left.",John,Bill,0
7,35F6NGNVMAX09EZJMTW59V8DGV67TW-1,good,ofalanal,ADJ,JJ,_,The meaning of ofalanal is having desirable or positive qualities especially those suitable for a thing specified.,"I couldn't decide if I should pressure cook or smoke the pig feet. After looking outside, I decided the weather was ofalanal enough to _ them.",smoke,pressure cook,0
8,67,drawer,ndurest,NOUN,NN,it,The word ndurest refers to a boxlike container in a piece of furniture; made so as to slide in and out.,"I used an old rag to clean the knife, and then I put _ in the ndurest.",The rag,The knife,1
9,185,unnecessary,cormantresy,ADJ,JJ,they,The meaning of cormantresy is not necessary.,Sam broke both his ankles and he's walking with crutches. But a month or so from now _ should be cormantresy.,The crutches,The ankles,0


Defining show one function to process training data - adjusting sentence and endings via dataset

In [147]:
def show_one(example):
    print(f"Context: {example['definition']}")
    print(f"  A - {example['sentence']} {example['ending0']}")
    print(f"  B - {example['sentence']} {example['ending1']}")
    print(f"\nGround truth: option {['A', 'B'][int(example['label'])]}")

In [148]:
show_one(dataset["train"][0])

Context: The meaning of nchubly is having or caused by an irregular surface.
  A - The floor started to have scratches when people's shoe started dragging on it. The _ was nchubly. floor
  B - The floor started to have scratches when people's shoe started dragging on it. The _ was nchubly. shoe

Ground truth: option B


In [149]:
show_one(dataset["train"][15])

Context: The meaning of loggetrid is having a low or inadequate temperature or feeling a sensation of coldness or having been made cold by e.g. ice or refrigeration.
  A - The doctor suggested alternating between ice and a heating pad when I strained a muscle, but he warned me to not let the _ get too loggetrid. pad
  B - The doctor suggested alternating between ice and a heating pad when I strained a muscle, but he warned me to not let the _ get too loggetrid. ice

Ground truth: option B


## Preprocessing the data

Preprocessing tokenized data from pretraining

In [150]:
from transformers import AutoTokenizer
from transformers import GPT2LMHeadModel


#tokenizer = AutoTokenizer.from_pretrained('gpt2', use_fast=True)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
#tokenizer = GPT2LMHeadModel.from_pretrained('gpt2')
#tokenizer = TFGPT2Tokenizer.from_pretrained("gpt2")

loading configuration file config.json from cache at /Users/zlxi/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /Users/zlxi/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loa

*Check fast tokenizer alternative to GPT2 model

Creating preprocess examples to handle initial multiple choice tasks

In [152]:
ending_names = ["ending0", "ending1"]

def preprocess_function(examples):
    # Repeat each first sentence four times to go with the four possibilities of second sentences.
    #first_sentences = [[context] * 2 for context in examples["sentence"]]
    # Grab all second sentences possible for each context.
    question_headers = examples["sentence"]
    second_sentences = [[f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)]
    
    # Flatten everything
    #first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    
    # Tokenize
    tokenized_examples = tokenizer(second_sentences, truncation=True)

    # Un-flatten
    return {k: [v[i:i+2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

Working with training length examples within the dataset

In [153]:
examples = dataset["train"][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

5 2 [26, 26]


Looking at decoded examples to verify input ids

In [154]:
idx = 1
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(2)]

["[CLS] the trophy doesn't fit into the brown suitcase because _ is too orherpt. the suitcase [SEP]",
 "[CLS] the trophy doesn't fit into the brown suitcase because _ is too orherpt. the trophy [SEP]"]

Comparing to ground truth label for evaluation

In [155]:
show_one(dataset["train"][1])

Context: The meaning of orherpt is above average in size or number or quantity or magnitude or extent.
  A - The trophy doesn't fit into the brown suitcase because _ is too orherpt. the suitcase
  B - The trophy doesn't fit into the brown suitcase because _ is too orherpt. the trophy

Ground truth: option B


Encoding datasets via the preprocess function

In [156]:
encoded_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/396 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

## Fine-tuning the model

Fine-tuning model for multiple choice

In [157]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer


from transformers import AutoConfig, AutoModel
model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

#usin gpt2
#config = AutoConfig.from_pretrained('gpt2')
#model = AutoModel.from_config(config)

#model = AutoModelForMultipleChoice.from_pretrained('GPT2Config')

loading configuration file config.json from cache at /Users/zlxi/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /Users/zlxi/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb42

Model checkpoint and instantiation

In [158]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-swag",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Using dataclass batch sizing

In [159]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}       
        #labels = [eval(i) for i in labels]
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

Flattening inputs/attentions masks etc. to  pass to the `tokenizer.pad` method. 

In [160]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
tokenizer.pad_token = tokenizer.mask_token
tokenizer.mask_token = ['PAD']
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
batch = DataCollatorForMultipleChoice(tokenizer)(features)

Assigning [PAD] to the pad_token key of the tokenizer
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Checking input ids 

In [161]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(2)]

['[CLS] the battery of the the phone died faster than the laptop battery, because the _ was always emusymb. phone [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] the battery of the the phone died faster than the laptop battery, because the _ was always emusymb. laptop [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']

In [162]:
show_one(dataset["train"][8])

Context: The word emusymb means in operation or operational.
  A - The battery of the the phone died faster than the laptop battery, because the _ was always emusymb. phone
  B - The battery of the the phone died faster than the laptop battery, because the _ was always emusymb. laptop

Ground truth: option A


Computing metrics for given predictions

In [163]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

Passing metrics to trainer

In [164]:
#from transformers import GPT2LMHeadModel
#model = GPT2LMHeadModel.from_pretrained("gpt2")

#model = AutoModelForMultipleChoice.from_pretrained('GPT2Config')

In [165]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/zlxi/Desktop/today-years-old/bert-base-uncased-finetuned-swag is already a clone of https://huggingface.co/zlxi/bert-base-uncased-finetuned-swag. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Finetuning via train method

In [166]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: ending0, pronoun, lemma, sentence, tag, definition, pos, ending1, id, fake_lemma. If ending0, pronoun, lemma, sentence, tag, definition, pos, ending1, id, fake_lemma are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/Users/zlxi/opt/anaconda3/envs/cs224n/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 396
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 75
  Number of trainable par

  0%|          | 0/75 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: ending0, pronoun, lemma, sentence, tag, definition, pos, ending1, id, fake_lemma. If ending0, pronoun, lemma, sentence, tag, definition, pos, ending1, id, fake_lemma are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.6945326924324036, 'eval_accuracy': 0.3799999952316284, 'eval_runtime': 11.5765, 'eval_samples_per_second': 8.638, 'eval_steps_per_second': 0.605, 'epoch': 1.0}


...

The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: ending0, pronoun, lemma, sentence, tag, definition, pos, ending1, id, fake_lemma. If ending0, pronoun, lemma, sentence, tag, definition, pos, ending1, id, fake_lemma are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16


  0%|          | 0/7 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.6935468912124634, 'eval_accuracy': 0.4099999964237213, 'eval_runtime': 13.6994, 'eval_samples_per_second': 7.3, 'eval_steps_per_second': 0.511, 'epoch': 3.0}
{'train_runtime': 538.9478, 'train_samples_per_second': 2.204, 'train_steps_per_second': 0.139, 'train_loss': 0.6970101928710938, 'epoch': 3.0}


TrainOutput(global_step=75, training_loss=0.6970101928710938, metrics={'train_runtime': 538.9478, 'train_samples_per_second': 2.204, 'train_steps_per_second': 0.139, 'train_loss': 0.6970101928710938, 'epoch': 3.0})